<a href="https://colab.research.google.com/github/zhenbinHU/popgenomics_R/blob/master/GWAS_maturity_group_soybean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# genome-wide association study for maturity group
# the data was downloaded from GRIN
# the SNP data was downloaded from soybase
# MLM from GAPIT was used

setwd("/homes/zhenbin/ft")
library(multtest)
library(gplots)
library(LDheatmap)
library(genetics)
library(ape)
library(EMMREML)
library(compiler) #this lib            
library("scatterplot3d")
source("/bulk/zhenbin/soybean_domestication/wj/G_E/gapit_functions_7.17.19.R")
source("http://zzlab.net/GAPIT/emma.txt")

hmp<-read.delim("/bulk/zhenbin/SOYBEAN/GWAS/soysnp50k_wm82.a2_41317.correct.hmp.txt",header=F)
id<-as.character(unlist(hmp[1,]))               

phen<-read.csv("maturity_group.csv",header=T,skip=2)
phen$Taxa<-paste0(phen$accession_prefix,phen$accession_number, phen$accession_surfix)

phen<-phen[,c("Taxa","observation_value")]
names(phen)<-c("Taxa","Maturity_Group")
phen$Maturity_Group<-as.character(phen$Maturity_Group)
mg<-c("0","00","000","I","II","III","IV","IX","V","VI","VII","VIII","X")

n<-0
for(i in mg){
    phen$Maturity_Group[phen$Maturity_Group==i]<-as.character(n)
    n<-n+1
}

phen$Maturity_Group<-as.numeric(phen$Maturity_Group)
phen<-aggregate(phen,by=list(phen$Taxa),mean)

phen<-phen[,c("Group.1","Maturity_Group")]
names(phen)<-c("Taxa","Maturity_Group")

pheno<-phen[which(phen$Taxa%in%id),]
snp<-hmp[,c(1:11,which(id%in%pheno$Taxa))]
snp$V3<-as.numeric(snp$V3)
snp[1,3]<-"chrom"
myGAPIT <- GAPIT(Y=pheno,G=snp,SNP.MAF=0.01,Model.selection=T)


# submit the job

#!/bin/bash
#SBATCH --job-name=gwas
#SBATCH --output=gwas.txt
#SBATCH --time=13-00:00:00
#SBATCH --mem-per-cpu=200GB
#SBATCH --mail-user=zhenbin@ksu.edu

cd /homes/zhenbin/ft
module load R

R CMD BATCH gwas
